In [39]:
!pip install pyspark

import kagglehub

# Download latest version
import kagglehub

# Download latest version
path = kagglehub.dataset_download("christofferms/pokemon-with-stats-and-image")

print("Path to dataset files:", path)

path = kagglehub.dataset_download("satvshr/top-4-used-car-sales-datasets-combined")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/christofferms/pokemon-with-stats-and-image/versions/1


100%|██████████| 446k/446k [00:00<00:00, 62.2MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/satvshr/top-4-used-car-sales-datasets-combined/versions/2


In [15]:
# Iniciar uma Sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploPySpark") \
    .getOrCreate()

In [21]:
# Imports
import pandas as pd
# Read the data with pandas
df_pandas = pd.read_csv("/root/.cache/kagglehub/datasets/christofferms/pokemon-with-stats-and-image/versions/1/pokedex.csv")
df_pandas.head(4)

,Image,Index,Name,Type 1,Type 2,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,images/1.png,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,images/2.png,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,images/3.png,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,images/4.png,3,Venusaur Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80


In [22]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ExemploLeitura").getOrCreate()

# Leitura de um arquivo CSV
df_pyspark = spark.read.csv("/root/.cache/kagglehub/datasets/christofferms/pokemon-with-stats-and-image/versions/1/pokedex.csv", header=True, inferSchema=True)

In [23]:
#Nomes das colunas
for i in df_pandas.columns:
  print(i)

Image
Index
Name
Type 1
Type 2
Total
HP
Attack
Defense
SP. Atk.
SP. Def
Speed


In [25]:
# Seleciona colunas específicas - Pandas
df_selecionado_pandas = df_pandas[['Name', 'Defense','Attack']]

In [26]:
# Seleciona colunas específicas - PySpark
df_selecionado_spark = df_pyspark.select('Name', 'Defense','Attack')

In [27]:
# Filtra linhas onde coluna1 > 50 - Pandas
df_filtrado_pandas = df_pandas[df_pandas['Total'] > 2]

In [28]:
# Filtra linhas onde coluna1 > 50 - PySpark
df_filtrado_spark = df_pyspark.filter(df_pyspark.Total > 2)

In [29]:
# Calcula a média de coluna1 agrupada por coluna2 - Pandas
df_agregado_pandas = df_pandas.groupby('Speed')['SP. Atk.'].mean().reset_index()

In [34]:
# Calcula a média de coluna1 agrupada por coluna2 - PySpark
from pyspark.sql.functions import avg

# Calcula a média de coluna1 agrupada por coluna2
df_agregado_spark = df_pyspark.groupBy("Speed").agg(avg("Attack").alias("media_Speed_Attack"))

In [35]:
df_selecionado_spark.show(5)
df_filtrado_spark.show(5)
df_agregado_spark.show(5)

+--------------------+-------+------+
|                Name|Defense|Attack|
+--------------------+-------+------+
|           Bulbasaur|     49|    49|
|             Ivysaur|     63|    62|
|            Venusaur|     83|    82|
|Venusaur Mega Ven...|    123|   100|
|          Charmander|     43|    52|
+--------------------+-------+------+
only showing top 5 rows

+------------+-----+--------------------+------+------+-----+---+------+-------+--------+-------+-----+
|       Image|Index|                Name|Type 1|Type 2|Total| HP|Attack|Defense|SP. Atk.|SP. Def|Speed|
+------------+-----+--------------------+------+------+-----+---+------+-------+--------+-------+-----+
|images/1.png|    1|           Bulbasaur| Grass|Poison|  318| 45|    49|     49|      65|     65|   45|
|images/2.png|    2|             Ivysaur| Grass|Poison|  405| 60|    62|     63|      80|     80|   60|
|images/3.png|    3|            Venusaur| Grass|Poison|  525| 80|    82|     83|     100|    100|   80|
|images/4

In [50]:
# Junção de dois DataFrames pandas
df1 = pd.read_csv("/root/.cache/kagglehub/datasets/christofferms/pokemon-with-stats-and-image/versions/1/pokedex.csv")
df2 = pd.read_csv("/root/.cache/kagglehub/datasets/satvshr/top-4-used-car-sales-datasets-combined/versions/2/output.csv")
df_concat = pd.concat([df1, df2], axis=1)

In [57]:
df1 = spark.read.csv("/root/.cache/kagglehub/datasets/christofferms/pokemon-with-stats-and-image/versions/1/pokedex.csv", header=True, inferSchema=True)
df1 = df1.withColumnRenamed("SP. Atk.", "SP_Atk")
df1 = df1.withColumnRenamed("Type 1", "type")
df_juncao = df1.join(df2, on="type", how="left")
df2 = spark.read.csv("/root/.cache/kagglehub/datasets/satvshr/top-4-used-car-sales-datasets-combined/versions/2/output.csv", header=True, inferSchema=True)

df_juncao = df1.join(df2, on="type", how="left")

df_juncao.show()

+-----+-------------+-----+--------------------+------+-----+---+------+-------+------+-------+-----+-----+-----+------------+----+----+------+----+-----+-----+--------+-------+-----+-----+
| type|        Image|Index|                Name|Type 2|Total| HP|Attack|Defense|SP_Atk|SP. Def|Speed|brand|model|transmission| age|fuel|engine|  km|owner|price|location|mileage|power|seats|
+-----+-------------+-----+--------------------+------+-----+---+------+-------+------+-------+-----+-----+-----+------------+----+----+------+----+-----+-----+--------+-------+-----+-----+
|Grass| images/1.png|    1|           Bulbasaur|Poison|  318| 45|    49|     49|    65|     65|   45| NULL| NULL|        NULL|NULL|NULL|  NULL|NULL| NULL| NULL|    NULL|   NULL| NULL| NULL|
|Grass| images/2.png|    2|             Ivysaur|Poison|  405| 60|    62|     63|    80|     80|   60| NULL| NULL|        NULL|NULL|NULL|  NULL|NULL| NULL| NULL|    NULL|   NULL| NULL| NULL|
|Grass| images/3.png|    3|            Venusaur|Po

In [59]:
# Cria uma nova coluna 'nova_coluna' como a soma de 'coluna1' e 'coluna2'
df_pandas['nova_coluna'] = df_pandas['Speed'] + df_pandas['Attack']

In [60]:
from pyspark.sql.functions import col

# Cria uma nova coluna 'nova_coluna' como a soma de 'coluna1' e 'coluna2'
df_pyspark = df_pyspark.withColumn("nova_coluna", col("Speed") + col("Attack"))

In [61]:
# Ordena o DataFrame pelo 'coluna1' em ordem descendente
df_pandas_ordenado = df_pandas.sort_values(by='Speed', ascending=False)

In [62]:
# Ordena o DataFrame pelo 'coluna1' em ordem descendente
df_pyspark_ordenado = df_pyspark.orderBy(df_pyspark.Speed.desc())